<a href="https://colab.research.google.com/github/naqi72/Finetuning_TTS_Model/blob/main/Hindi/HINDI_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
!pip install transformers datasets soundfile accelerate speechbrain


In [102]:
from huggingface_hub import notebook_login

notebook_login()


In [103]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [104]:
from datasets import load_dataset, Audio

dataset = load_dataset("1rsh/tts-rj-hi-karya")

In [76]:
dataset


DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 422603
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 4269
    })
})

In [105]:
dataset = dataset['train']


In [106]:
len(dataset)


422603

In [107]:
half_size = len(dataset) // 75
dataset = dataset.select(range(half_size))

print(dataset)


Dataset({
    features: ['audio', 'sentence'],
    num_rows: 5634
})


In [108]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))


In [109]:
from transformers import SpeechT5Processor

model_name = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(model_name)


In [110]:
tokenizer = processor.tokenizer


In [111]:
dataset[2:5]


{'audio': [{'path': '6006_080.mp3',
   'array': array([-6.19105995e-05, -2.88616866e-05,  4.09726053e-05, ...,
           7.02164616e-05,  1.13365586e-05, -1.18132175e-05]),
   'sampling_rate': 16000},
  {'path': '0126_001.mp3',
   'array': array([-0.00010195, -0.00016777, -0.00025085, ..., -0.07432213,
          -0.0553013 , -0.02470658]),
   'sampling_rate': 16000},
  {'path': '2705_072.mp3',
   'array': array([-2.76818882e-05, -1.68277966e-05, -4.99295493e-06, ...,
           2.49382720e-05,  2.69935372e-05,  1.48971012e-05]),
   'sampling_rate': 16000}],
 'sentence': ['जुम्मन बोले-यह अलगू चौधरी की इच्छा पर निर्भर है।\n',
  'पोखर के रास्ते में उसे चोरु मिल गया|\n',
  'की दूरी पर नागरकोविल है।\n']}

In [112]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}


Map:   0%|          | 0/5634 [00:00<?, ? examples/s]

In [113]:
dataset_vocab - tokenizer_vocab


{'\n',
 ' ',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '|',
 'ँ',
 'ं',
 'ः',
 'अ',
 'आ',
 'इ',
 'ई',
 'उ',
 'ऊ',
 'ऋ',
 'ऍ',
 'ए',
 'ऐ',
 'ऑ',
 'ओ',
 'औ',
 'क',
 'ख',
 'ग',
 'घ',
 'च',
 'छ',
 'ज',
 'झ',
 'ञ',
 'ट',
 'ठ',
 'ड',
 'ढ',
 'ण',
 'त',
 'थ',
 'द',
 'ध',
 'न',
 'प',
 'फ',
 'ब',
 'भ',
 'म',
 'य',
 'र',
 'ल',
 'व',
 'श',
 'ष',
 'स',
 'ह',
 '़',
 'ा',
 'ि',
 'ी',
 'ु',
 'ू',
 'ृ',
 'े',
 'ै',
 'ॉ',
 'ो',
 'ौ',
 '्',
 'क़',
 'ख़',
 'ग़',
 'ज़',
 'ड़',
 'ढ़',
 'फ़',
 'ॠ',
 '।',
 '०',
 '१',
 '२',
 '३',
 '८',
 '९',
 '\u200b',
 '\u200d',
 '‘',
 '’',
 '…'}

In [114]:
import re

def normalize_text(text):

    # Remove punctuation
    text = re.sub(r'[^\u0900-\u097F\s\']', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Define a function to add the normalized_text column
def add_normalized_text(example):
    example['normalized_text'] = normalize_text(example['sentence'])
    return example

# Apply the function to the dataset
dataset = dataset.map(add_normalized_text)

# Print the first few examples to verify
print(dataset[2:5])


Map:   0%|          | 0/5634 [00:00<?, ? examples/s]

{'audio': [{'path': '6006_080.mp3', 'array': array([-6.19105995e-05, -2.88616866e-05,  4.09726053e-05, ...,
        7.02164616e-05,  1.13365586e-05, -1.18132175e-05]), 'sampling_rate': 16000}, {'path': '0126_001.mp3', 'array': array([-0.00010195, -0.00016777, -0.00025085, ..., -0.07432213,
       -0.0553013 , -0.02470658]), 'sampling_rate': 16000}, {'path': '2705_072.mp3', 'array': array([-2.76818882e-05, -1.68277966e-05, -4.99295493e-06, ...,
        2.49382720e-05,  2.69935372e-05,  1.48971012e-05]), 'sampling_rate': 16000}], 'sentence': ['जुम्मन बोले-यह अलगू चौधरी की इच्छा पर निर्भर है।\n', 'पोखर के रास्ते में उसे चोरु मिल गया|\n', 'की दूरी पर नागरकोविल है।\n'], 'normalized_text': ['जुम्मन बोलेयह अलगू चौधरी की इच्छा पर निर्भर है।', 'पोखर के रास्ते में उसे चोरु मिल गया', 'की दूरी पर नागरकोविल है।']}


In [115]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}


Map:   0%|          | 0/5634 [00:00<?, ? examples/s]

In [116]:
new_symbols = ['४', '५', '६','७']
dataset_vocab.update(new_symbols)

In [117]:
dataset_vocab - tokenizer_vocab


{' ',
 'ँ',
 'ं',
 'ः',
 'अ',
 'आ',
 'इ',
 'ई',
 'उ',
 'ऊ',
 'ऋ',
 'ऍ',
 'ए',
 'ऐ',
 'ऑ',
 'ओ',
 'औ',
 'क',
 'ख',
 'ग',
 'घ',
 'च',
 'छ',
 'ज',
 'झ',
 'ञ',
 'ट',
 'ठ',
 'ड',
 'ढ',
 'ण',
 'त',
 'थ',
 'द',
 'ध',
 'न',
 'प',
 'फ',
 'ब',
 'भ',
 'म',
 'य',
 'र',
 'ल',
 'व',
 'श',
 'ष',
 'स',
 'ह',
 '़',
 'ा',
 'ि',
 'ी',
 'ु',
 'ू',
 'ृ',
 'े',
 'ै',
 'ॉ',
 'ो',
 'ौ',
 '्',
 'क़',
 'ख़',
 'ग़',
 'ज़',
 'ड़',
 'ढ़',
 'फ़',
 'ॠ',
 '।',
 '०',
 '१',
 '२',
 '३',
 '४',
 '५',
 '६',
 '७',
 '८',
 '९'}

In [118]:
replacements = {
    ('ँ', 'n'),
    ('ं', 'n'),
    ('ः', 'h'),
    ('अ', 'uh'),
    ('आ', 'aa'),
    ('इ', 'i'),
    ('ई', 'ee'),
    ('उ', 'u'),
    ('ऊ', 'oo'),
    ('ऋ', 'ri'),
    ('ऍ', 'ae'),
    ('ए', 'e'),
    ('ऐ', 'ai'),
    ('ऑ', 'aw'),
    ('ओ', 'o'),
    ('औ', 'au'),
    ('क', 'k'),
    ('ख', 'kh'),
    ('ग', 'g'),
    ('घ', 'gh'),
    ('च', 'ch'),
    ('छ', 'chh'),
    ('ज', 'j'),
    ('झ', 'jh'),
    ('ञ', 'ny'),
    ('ट', 't'),
    ('ठ', 'th'),
    ('ड', 'd'),
    ('ढ', 'dh'),
    ('ण', 'n'),
    ('त', 't'),
    ('थ', 'th'),
    ('द', 'd'),
    ('ध', 'dh'),
    ('न', 'n'),
    ('प', 'p'),
    ('फ', 'ph'),
    ('ब', 'b'),
    ('भ', 'bh'),
    ('म', 'm'),
    ('य', 'y'),
    ('र', 'r'),
    ('ल', 'l'),
    ('व', 'v'),
    ('श', 'sh'),
    ('ष', 'shh'),
    ('स', 's'),
    ('ह', 'h'),
    ('़', ''),
    ('ा', 'aa'),
    ('ि', 'i'),
    ('ी', 'ee'),
    ('ु', 'u'),
    ('ू', 'oo'),
    ('ृ', 'ri'),
    ('े', 'e'),
    ('ै', 'ai'),
    ('ॉ', 'aw'),
    ('ो', 'o'),
    ('ौ', 'au'),
    ('्', ''),
    ('क़', 'q'),
    ('ख़', 'kh'),
    ('ग़', 'gh'),
    ('ज़', 'z'),
    ('ड़', 'r'),
    ('ढ़', 'rh'),
    ('फ़', 'f'),
    ('ॠ', 'rri'),
    ('।', 'period'),
    ('०', '0'),
    ('१', '1'),
    ('२', '2'),
    ('३', '3'),
    ('४', '4'),
    ('५', '5'),
    ('६', '6'),
    ('७', '7'),
    ('८', '8'),
    ('९', '9')
}


In [119]:
def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
    return inputs

dataset = dataset.map(cleanup_text)


Map:   0%|          | 0/5634 [00:00<?, ? examples/s]

In [120]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings


/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [121]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example


In [122]:
processed_example = prepare_dataset(dataset[0])
list(processed_example.keys())


['input_ids', 'labels', 'speaker_embeddings']

In [123]:
processed_example["speaker_embeddings"].shape


(512,)

In [124]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)


Map:   0%|          | 0/5634 [00:00<?, ? examples/s]

In [125]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)


Filter:   0%|          | 0/5634 [00:00<?, ? examples/s]

5634

In [126]:
dataset = dataset.train_test_split(test_size=0.1)


In [127]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch


In [128]:
data_collator = TTSDataCollatorWithPadding(processor=processor)


In [129]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(model_name)


In [133]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="speecht5_finetuned_rohit_hindi",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [134]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [135]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss
100,1.398900,1.026405
200,0.942100,0.795459
300,0.851000,0.711958
400,0.796600,0.691743
500,0.771900,0.678817


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1876}
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)`

TrainOutput(global_step=500, training_loss=1.040471321105957, metrics={'train_runtime': 959.9992, 'train_samples_per_second': 16.667, 'train_steps_per_second': 0.521, 'total_flos': 923124532131648.0, 'train_loss': 1.040471321105957, 'epoch': 3.1545741324921135})

In [137]:
trainer.push_to_hub()


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1876}
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/naqi72/speecht5_finetuned_rohit_hindi/commit/2e6596fa2cf9c8a76560b278220eef6bfe222b42', commit_message='End of training', commit_description='', oid='2e6596fa2cf9c8a76560b278220eef6bfe222b42', pr_url=None, pr_revision=None, pr_num=None)

In [151]:
from tqdm import tqdm


In [152]:
# Step 5: Evaluating the Model using MOS
def generate_speech_and_collect_mos(model, tokenizer, eval_dataset):
    mos_scores = []
    for index, row in tqdm(eval_dataset.iterrows(), total=len(eval_dataset)):
        input_ids = row['input_ids']
        audio_output = model.generate(input_ids=input_ids)

        # Save or play the audio, then have native speakers rate the speech quality
        mos = collect_mos_from_user(audio_output)  # Custom function for human evaluation
        mos_scores.append(mos)

    return np.mean(mos_scores)




In [153]:
# Placeholder for MOS collection function
def collect_mos_from_user(audio):
    # Play the audio to a native speaker and get their rating
    mos = np.random.uniform(1, 5)  # Placeholder for MOS score (random generation)
    return mos

In [138]:
model = SpeechT5ForTextToSpeech.from_pretrained(
    "naqi72/Finetune_Hindi_SpeechT5"
)


config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [160]:
example = dataset["test"][304]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)
speaker_embeddings.shape

torch.Size([1, 512])

In [140]:
text = "उम्मीदवार ने समझाया कि CUDA GPU प्रदर्शन को कैसे बढ़ाता है और OAuth API एक्सेस को कैसे सुरक्षित करता है।"


In [141]:
!pip install indic-num2words

from num_to_words import num_to_word

import re

def replace_numbers_with_words(text):
    def replace(match):
        number = int(match.group())
        return number_to_word(number, lang='hi')

    result = re.sub(r'\b\d+\b', replace, text)
    return result

In [142]:
def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [167]:
text

'उम्मीदवार ने समझाया कि CUDA GPU प्रदर्शन को कैसे बढ़ाता है और OAuth API एक्सेस को कैसे सुरक्षित करता है।'

In [178]:
converted_text = replace_numbers_with_words(text)
cleaned_text=cleanup_text(converted_text)

#final_text = normalize_text(cleaned_text)
#final_text


In [191]:
from transformers import SpeechT5HifiGan
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")


In [179]:
inputs = processor(text=cleaned_text, return_tensors="pt")


In [185]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)


In [190]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)


In [158]:
sf.write('output.wav', speech.numpy(), 16000)


In [ ]:
import time
start_time = time.time()

# Run inference
from IPython.display import Audio
Audio(speech.numpy(), rate=16000)

# Stop the timer
end_time = time.time()

# Calculate inference time
inference_time = end_time - start_time
print(f"Inference Time: {inference_time:.4f} seconds")
